In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import io
from collections import defaultdict
import codecs

In [3]:
# List data
data_path =  "/content/drive/MyDrive/School Projects/CS221 - Xử lý ngôn ngữ tự nhiên/Data"
file_name = os.listdir(data_path)
file_path = [os.path.join(data_path, name) for name in file_name]

In [15]:
data_hard = io.open(file_path[0], encoding="utf-8").readlines()
data_easy = io.open(file_path[1], encoding="utf-8").readlines()

In [16]:
data = data_easy
data.extend(data_hard)

In [60]:
tagsetDict = {
    "N": 0,
    "V": 1,
    "A": 2,
    "P": 3,
    "M": 4,
    "D": 5,
    "R": 6,
    "E": 7,
    "C": 8,
    "I": 9,
    "X": 10
}

In [25]:
inverseTagsetDict = {tagsetDict[k]: k for k in tagsetDict}
wordBank = defaultdict()
bigramBank = defaultdict()
bigramDict = defaultdict(int)

In [26]:
for line in train:
    line_split = line.split()
    for i, w in enumerate(line_split):
        parts = w.split("/")
        if i >= 1:
            prevParts = line_split[i-1].split("/")
        if len(parts) == 1 or parts[1] not in tagsetDict:
            continue

        word = parts[0]
        pos = parts[1]
        if i >= 1:
            prevWord = prevParts[0]
            prevPos = prevParts[1]
            bigramBank[word] = (pos, prevWord, prevPos)
            bigramDict[(pos, prevPos)] += 1

        if word not in wordBank:
            wordBank[word] = [pos]
        else:
            wordBank[word] += [pos]

In [27]:
bi_grams = []
tri_grams = []

for sentence in set(wordBank):
    temp = 0
    for s in list(sentence):
        if s == "_":
            temp += 1
    if temp == 1:
        bi_grams.append(sentence.replace("_", " "))
    elif temp == 2:
        tri_grams.append(sentence.replace("_", " "))

In [30]:
bigramFreq = {x: [0]*10 for x in tagsetDict}
mostCommonBigrams = defaultdict()
for k in bigramFreq:
    maxFreq = 0
    maxPos = "N"
    for i, x in enumerate(bigramFreq[k]):
        if x > maxFreq:
            maxFreq = x
            maxPos = inverseTagsetDict[i]
    mostCommonBigrams[k] = maxPos

In [31]:
def Viterbi_rule_based(word, wordIdx, lineSize, line):
    feat = [1]
    sentPercent = float(wordIdx)/float(lineSize)
    feat.append(sentPercent)

    if word[0].isupper() and wordIdx != 0:
        feat.append(1)
    else:
        feat.append(0)

    posIdx_array = ([0] * len(tagsetDict))
    posSet = []
    if word in wordBank:
        posSet = wordBank[word]
    else:
        if wordIdx == 0:
            posSet = list(tagsetDict.keys())[0]
            posIdx_array[tagsetDict[posSet]] = 1
            return feat + posIdx_array + [0]
        else:
            prevWord = line[wordIdx-1]
            if prevWord in wordBank:
                prevPos = wordBank[prevWord]
                maxPos = mostCommonBigrams[prevPos]
                if prevPos == "E":
                    maxPos = 3
                posIdx_array[tagsetDict[maxPos]] = 1
                feat += posIdx_array + [tagsetDict[maxPos]]
                return feat
            else:
                posSet = list(tagsetDict.keys())[0]
                posIdx_array[tagsetDict[posSet]] = 1
                feat += posIdx_array + [0]
                return feat

    for pos in posSet:
        posIdx = tagsetDict[pos]
        posIdx_array[posIdx] += 1.0 / len(wordBank[word])
    feat += (posIdx_array) + [0]
    return feat

In [32]:
y = []
X_train = []
for line in train:
    l_split = line.split()
    for i, w in enumerate(l_split):
        parts = w.split("/")
        word = parts[0]
        len_line = len(l_split)

        if len(parts) == 1 or parts[1] not in tagsetDict:
            continue
        y.append(wordBank[word][0])
        X_train.append(Viterbi_rule_based(word, i, len_line, l_split))

In [33]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [34]:
train_fit = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y)

In [35]:
import re
import unicodedata as ud

In [43]:
import string

In [46]:
X_test = []
correct_results = []
for line in test:
    l_split = line.split()
    for i,w in enumerate(l_split):
        len_line = len(l_split)
        parts = w.split("/")
        if len(parts) == 1 or parts[1] not in tagsetDict:
            continue
        word = parts[0]
        pos = parts[1]
        X_test.append(Viterbi_rule_based(word,i,len_line,l_split))
        correct_results.append(pos)

In [52]:
predicted_results = train_fit.predict(X_test)

Viterbi Algorithm Accuracy: 64.13793103448275


In [51]:
from sklearn.metrics import classification_report
print('Results of the Markov hidden model combined with the Viterbi algorithm:\n')
print(classification_report(predicted_results, correct_results))

Results of the Markov hidden model combined with the Viterbi algorithm:

              precision    recall  f1-score   support

           A       0.12      1.00      0.21         2
           C       0.57      1.00      0.73         4
           D       0.80      1.00      0.89         4
           E       0.70      0.88      0.78         8
           I       0.00      0.00      0.00         0
           M       0.50      1.00      0.67         1
           N       0.98      0.48      0.65        85
           P       0.80      0.92      0.86        13
           R       0.82      0.90      0.86        10
           V       0.37      0.76      0.50        17
           X       0.00      0.00      0.00         1

    accuracy                           0.64       145
   macro avg       0.51      0.72      0.56       145
weighted avg       0.83      0.64      0.67       145



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

In [1]:
wl = ["vì","nó","rất","đặc_biệt","nên","tôi","đã","chú_ý"]

In [54]:
for i in wl:
    print(toString(i), end='')

vì/E nó/P rất/R đặc_biệt/A nên/C tôi/P đã/R chú_ý/V 